In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("../data/mockdata.csv")
df

,age,housingtype,yearly income,cpf balance,yearly expenditure,savings,quality of life,disaster preparedness,retirement readiness
0,39,3-Room Flat,176846,120000,63000,9475,8,3,2
1,27,5-Room,83068,150000,34000,6280,8,2,3
2,53,5-Room,99755,300000,24000,116916,7,9,1
3,25,Landed Property,198879,70000,177000,8050,7,4,5
4,34,4-Room Flat,97753,180000,57000,557,7,5,4
...,...,...,...,...,...,...,...,...,...
4995,49,Apartment,25227,300000,11000,473221,10,10,5
4996,59,1&2-Room Flat,124741,210000,20000,230207,9,7,1
4997,34,1&2-Room Flat,51688,150000,13000,8264,8,5,1
4998,58,Apartment,106048,200000,23000,145291,6,8,4


In [3]:
# Encode the housingtype
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()

df[["housingtype"]] = encoder.fit_transform(df[["housingtype"]])
df["housingtype"]

0       1.0
1       3.0
2       3.0
3       7.0
4       2.0
       ... 
4995    4.0
4996    0.0
4997    0.0
4998    4.0
4999    0.0
Name: housingtype, Length: 5000, dtype: float64

In [4]:
df.describe()

,age,housingtype,yearly income,cpf balance,yearly expenditure,savings,quality of life,disaster preparedness,retirement readiness
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,47.474200,3.500600,112094.894800,263110.000000,60965.800000,199050.220800,7.005400,6.161800,3.267400
std,13.351097,2.273551,50913.879436,142376.690945,41907.401213,186064.809204,2.235883,2.814108,1.793137
min,25.000000,0.000000,24021.000000,10000.000000,10000.000000,500.000000,1.000000,1.000000,1.000000
25%,36.000000,2.000000,68346.250000,140000.000000,26000.000000,7217.750000,6.000000,4.000000,2.000000
50%,47.000000,3.000000,111664.000000,270000.000000,50000.000000,171698.500000,7.000000,7.000000,3.000000
75%,59.000000,5.000000,156897.500000,380000.000000,86000.000000,359344.750000,9.000000,9.000000,4.000000
max,70.000000,7.000000,199929.000000,500000.000000,195000.000000,657783.000000,10.000000,10.000000,10.000000


In [5]:
from sklearn.model_selection import train_test_split

# Split the data into features (X) and target (y)
X = df.drop(columns=["quality of life", "disaster preparedness", "retirement readiness"])
y = df["quality of life"]
# y = df["disaster preparedness"]
# y = df["retirement readiness"]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [6]:
# Since the standard deviation between each data point is high, we need to standardise/normalise it
# Since we have outliers & n>30 so CLT -> norm dist, we just choose standardisation over normalisation.

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
# Now we implement the model.
from sklearn import svm
from sklearn.model_selection import GridSearchCV

param_grid = {
    "C": np.logspace(-2, 4, num=7),
    "gamma": np.logspace(-3, 2, num=6)
}

svr = svm.SVR()

grid = GridSearchCV(
    svr,
    param_grid=param_grid,
    scoring="neg_mean_absolute_error"
)

grid.fit(X_train, y_train)

TypeError: Parameter grid for parameter 'C' needs to be a list or a numpy array, but got 10000.0 (of type float) instead. Single values need to be wrapped in a list with one element.

In [ ]:
grid.best_params_

{'C': 10000.0, 'gamma': 0.01}

In [ ]:
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, accuracy_score

# Predict values
y_pred = grid.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)

rmse = root_mean_squared_error(y_test, y_pred)
print("RMSE:", rmse)
mean_y = sum(list(y))/len(y)
print("Mean of target value:", mean_y )
# While not a true metric, we calculate the % as a gauge.
print("Accuracy: ", accuracy_score(y_pred, y_test))

MAE 1.6167373601819408
RMSE 1.914967263649654
Mean of target value: 7.0054
27.3355877415944
